# Clean webtext from charter school data

- Author: Jaren Haber, Madeleine Peng, James Jung
- Institution: UC Berkeley
- Date created: July 26, 2019
- Date last edited: 

Description: 

## Initialize

In [1]:
# Import packages
import pandas as pd # For working with DataFrames
import gc # To accelerate loading pickle files

# Show visualizations within notebook:
%matplotlib inline 

In [2]:
# Load functions from data_tools directory:
import sys; sys.path.insert(0, "../tools")

# For displaying basic DF info, storing DFs for memory efficiency, and loading a filtered DF:
from df_tools import check_df, convert_df, load_filtered_df, replace_df_nulls

# For quickly loading & saving pickle files in Python:
from quickpickle import quickpickle_dump, quickpickle_load 

# For saving and loading text lists to/from file:
from textlist_file import write_list, load_list 

In [3]:
# Set file paths
charters_path = "../../nowdata/charters_2015.pkl"
original_path = "../../nowdata/backups/charters_full_2015_250_v2a_unlappedtext_counts3_geoleaid.pkl"
filtered_path = "../../nowdata/parsing/filtered_10.pkl"
raw_folder = "../../nowdata/webtext_raw/" # for raw webtext when extracted
raw_filtered_data = raw_folder + "webtext_unlapped_filtered_10.pkl"
cleaned_folder = "../../nowdata/webtext_cleaned/" # for cleaned webtext: save as CSV, include ONLY the columns "NCESSCH" (unique school identifier) and "text_full" (renamed from "WEBTEXT")

## Define helper function(s)

In [ ]:
def clean_webtext(ls):
    
    '''This function cleans and tokenizes sentences, removing punctuation and numbers and making words into lower-case stems.
    Inputs: list of strings;
    This function loops over all elements in the input list given, cleans the texts and returns one string'''
        
    global mpdo # Check if we're doing multiprocessing. If so, then mpdo=True
    global sents_combined # Grants access to variable holding a list of lists of words, where each list of words represents a sentence in its original order (only relevant for this function if we're not using multiprocessing)
    global pcount # Grants access to preprocessing counter
    
    known_pages = set() # Initialize list of known pages for a school
    sents_combined = [] # Initialize list of all school's sentences
    
    #print('Parsing school #' + str(pcount)) # Print number of school being parsed

    for s in ls: # Iterate over tuples in tuplist (list of tuples)
        for chunk in s.split('\n'): 
            for sent in sent_tokenize(chunk): # Tokenize chunk by sentences (in case >1 sentence in chunk)
                #sent = clean_sentence(sent, fast=True) # Clean and tokenize sentence
                sent = clean_sentence(sent)
                if ((sent == []) or (len(sent) == 0)): # If sentence is empty, continue to next sentence without appending
                    continue
                
                sents_combined.extend(sent) # add sent to school object
                
    school_sentslist.append(sents_combined) # add list of sentence to full list 
    
    return sents_combined

## Clean webtext

In [9]:
# Load data
df = quickpickle_load(raw_filtered_data)[["text_full", "NCESSCH"]]
check_df(df, "NCESSCH")

# rows and cols:  (6103, 2)
# duplicates by NCESSCH: 0

Columns and # missing cases (if any): 
text_full
NCESSCH


In [ ]:
ls_str = [] #initialize list for concatenated string for webtext per school

for i in range(len(df)):
    ls_str.append(' '.join(preprocess_wem2(df['WEBTEXT'][i])))
    if i%500 == 0:
        print("Cleaned ", i, " rows")
    
df['text_full'] = ls_str

## Next step: Merging!
Now merge this cleaned text with the covariates from the previous version of the charter schools data (change only the `text_full` column).